In [0]:

dfCountries = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("dbfs:/FileStore/tables/landing/countries.csv")
dfStates = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("dbfs:/FileStore/tables/landing/states.csv")
dfBreweries = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/landing/breweries.csv")

In [0]:

dfCountries.write.mode('overwrite').csv('dbfs:/FileStore/tables/bootcampociandt/DimCountries')
dfCountries.show()

+----+-------------------+
|code|       country_name|
+----+-------------------+
|  AF|        Afghanistan|
|  AX|      Åland Islands|
|  AL|            Albania|
|  DZ|            Algeria|
|  AS|     American Samoa|
|  AD|            Andorra|
|  AO|             Angola|
|  AI|           Anguilla|
|  AQ|         Antarctica|
|  AG|Antigua and Barbuda|
|  AR|          Argentina|
|  AM|            Armenia|
|  AW|              Aruba|
|  AU|          Australia|
|  AT|            Austria|
|  AZ|         Azerbaijan|
|  BH|            Bahrain|
|  BS|            Bahamas|
|  BD|         Bangladesh|
|  BB|           Barbados|
+----+-------------------+
only showing top 20 rows



In [0]:

dfStates.write.mode('overwrite').csv('dbfs:/FileStore/tables/bootcampociandt/DimStates')
dfStates.show()

+----+--------------------+
|code|          state_name|
+----+--------------------+
|  AL|             Alabama|
|  AK|              Alaska|
|  AS|      American Samoa|
|  AZ|             Arizona|
|  AR|            Arkansas|
|  CA|          California|
|  CO|            Colorado|
|  CT|         Connecticut|
|  DE|            Delaware|
|  DC|District Of Columbia|
|  FL|             Florida|
|  GA|             Georgia|
|  GU|                Guam|
|  HI|              Hawaii|
|  ID|               Idaho|
|  IL|            Illinois|
|  IN|             Indiana|
|  IA|                Iowa|
|  KS|              Kansas|
|  KY|            Kentucky|
+----+--------------------+
only showing top 20 rows



In [0]:
dfBreweries.show()

+-----+--------------------+--------------------+-----+-------+--------------------+--------------------+
|   id|                name|                city|state|country|               notes|               types|
+-----+--------------------+--------------------+-----+-------+--------------------+--------------------+
|19730|     Brouwerij Danny|           Erpe-Mere| null|     BE|No notes at this ...|             Brewery|
|32541|Coachella Valley ...|      Thousand Palms|   CA|     US|No notes at this ...|Brewery, Bar, Bee...|
|44736|    Beef 'O' Brady's|          Plant City|   FL|     US|No notes at this ...|         Bar, Eatery|
|23372|Broadway Wine Mer...|       Oklahoma City|   OK|     US|No notes at this ...|               Store|
|35328|Brighton Beer Dis...|            Brighton|  GB2|     GB|Duplicate of http...|         Bar, Eatery|
|31561|      Teddy's Tavern|             Seattle|   WA|     US|No notes at this ...|     Bar, Beer-to-go|
|35975|Modus Operandi Br...|           Mona Va

In [0]:
from pyspark.sql.functions import split, col, count, upper, trim, substring, isnan, when, countDistinct
from pyspark.sql.functions import *
from pyspark.sql.window import Window

dfTrataTypes = dfBreweries.select('id', 'name','types') \
.withColumn('type', explode(split('types',','))) \
.drop('types')
dfTrataTypes.show()

+-----+--------------------+-----------+
|   id|                name|       type|
+-----+--------------------+-----------+
|19730|     Brouwerij Danny|    Brewery|
|32541|Coachella Valley ...|    Brewery|
|32541|Coachella Valley ...|        Bar|
|32541|Coachella Valley ...| Beer-to-go|
|44736|    Beef 'O' Brady's|        Bar|
|44736|    Beef 'O' Brady's|     Eatery|
|23372|Broadway Wine Mer...|      Store|
|35328|Brighton Beer Dis...|        Bar|
|35328|Brighton Beer Dis...|     Eatery|
|31561|      Teddy's Tavern|        Bar|
|31561|      Teddy's Tavern| Beer-to-go|
|35975|Modus Operandi Br...|    Brewery|
|35975|Modus Operandi Br...|        Bar|
|35975|Modus Operandi Br...|     Eatery|
|35975|Modus Operandi Br...| Beer-to-go|
| 5618|Hops! Beer Restau...|    Brewery|
| 5618|Hops! Beer Restau...|        Bar|
| 5618|Hops! Beer Restau...|     Eatery|
|30916|     Kelly's Cellars|        Bar|
|41278|       The Other End|        Bar|
+-----+--------------------+-----------+
only showing top

In [0]:
dfTrataTypes

Out[6]: DataFrame[id: string, name: string, type: string]

In [0]:
dfAggTypes = dfTrataTypes.groupBy(trim(upper('type')).alias('Type')) \
  .agg(count('*').alias('count')) \
  .filter(col('count')>1)
dfAggTypes.show()
dfAggTypes.count()


+----------+-----+
|      Type|count|
+----------+-----+
|  HOMEBREW| 1351|
|    EATERY|22849|
|     STORE| 9725|
|       BAR|29515|
|   BREWERY|18929|
|BEER-TO-GO|10638|
+----------+-----+

Out[7]: 6

In [0]:


df1=dfAggTypes.select('Type').rdd.zipWithIndex().toDF()
dfDimTypes=df1.select( col("_1.*") , (col("_2")+1).alias("cod") ) 
dfDimTypes.show()
#dfDimTpes = dfAggTypes.

+----------+---+
|      Type|cod|
+----------+---+
|  HOMEBREW|  1|
|    EATERY|  2|
|     STORE|  3|
|       BAR|  4|
|   BREWERY|  5|
|BEER-TO-GO|  6|
+----------+---+



In [0]:
dfDimTypes.write.mode('overwrite').csv('dbfs:/FileStore/tables/bootcampociandt/DimTypes')

In [0]:
dfBrewerryTypesAux = dfTrataTypes.drop('name')
dfBrewerryTypesAux.write.mode('overwrite').csv('dbfs:/FileStore/tables/bootcampociandt/BrewerryTypesAux')
dfBrewerryTypesAux.show()

+-----+-----------+
|   id|       type|
+-----+-----------+
|19730|    Brewery|
|32541|    Brewery|
|32541|        Bar|
|32541| Beer-to-go|
|44736|        Bar|
|44736|     Eatery|
|23372|      Store|
|35328|        Bar|
|35328|     Eatery|
|31561|        Bar|
|31561| Beer-to-go|
|35975|    Brewery|
|35975|        Bar|
|35975|     Eatery|
|35975| Beer-to-go|
| 5618|    Brewery|
| 5618|        Bar|
| 5618|     Eatery|
|30916|        Bar|
|41278|        Bar|
+-----+-----------+
only showing top 20 rows



In [0]:
 dfBrewery = dfBreweries.withColumn('state', coalesce(substring(trim('state'), 0, 2), lit('--') )) \
.withColumn('city', upper(trim('city'))) \
.withColumn('name', upper(trim('name'))) \
.drop('notes','types') 
dfBrewery.show()

+-----+--------------------+--------------------+-----+-------+
|   id|                name|                city|state|country|
+-----+--------------------+--------------------+-----+-------+
|19730|     BROUWERIJ DANNY|           ERPE-MERE|   --|     BE|
|32541|COACHELLA VALLEY ...|      THOUSAND PALMS|   CA|     US|
|44736|    BEEF 'O' BRADY'S|          PLANT CITY|   FL|     US|
|23372|BROADWAY WINE MER...|       OKLAHOMA CITY|   OK|     US|
|35328|BRIGHTON BEER DIS...|            BRIGHTON|   GB|     GB|
|31561|      TEDDY'S TAVERN|             SEATTLE|   WA|     US|
|35975|MODUS OPERANDI BR...|           MONA VALE|   --|     AU|
| 5618|HOPS! BEER RESTAU...|       RICCIONE (RN)|   --|     IT|
|30916|     KELLY'S CELLARS|             BELFAST|   GB|     GB|
|41278|       THE OTHER END|              DESTIN|   FL|     US|
|31711|    TEN BELLS TAVERN|              DALLAS|   TX|     US|
|12324|BISTRO EUROPA / H...|          ALEXANDRIA|   VA|     US|
|49850|  CRAFTY'S BEER SHOP|LETCHWORTH G

In [0]:
dfBrewery.write.mode('overwrite').csv('dbfs:/FileStore/tables/bootcampociandt/Brewery')

In [0]:
KeyJoin = (dfBrewery.country == dfCountries.code )
dfBreweryCountry =  dfBrewery.join(dfCountries, KeyJoin, 'left') \
.drop('code')
dfBreweryCountry.show()

+-----+--------------------+--------------------+-----+-------+--------------+
|   id|                name|                city|state|country|  country_name|
+-----+--------------------+--------------------+-----+-------+--------------+
|19730|     BROUWERIJ DANNY|           ERPE-MERE|   --|     BE|       Belgium|
|32541|COACHELLA VALLEY ...|      THOUSAND PALMS|   CA|     US| United States|
|44736|    BEEF 'O' BRADY'S|          PLANT CITY|   FL|     US| United States|
|23372|BROADWAY WINE MER...|       OKLAHOMA CITY|   OK|     US| United States|
|35328|BRIGHTON BEER DIS...|            BRIGHTON|   GB|     GB|United Kingdom|
|31561|      TEDDY'S TAVERN|             SEATTLE|   WA|     US| United States|
|35975|MODUS OPERANDI BR...|           MONA VALE|   --|     AU|     Australia|
| 5618|HOPS! BEER RESTAU...|       RICCIONE (RN)|   --|     IT|         Italy|
|30916|     KELLY'S CELLARS|             BELFAST|   GB|     GB|United Kingdom|
|41278|       THE OTHER END|              DESTIN|   

In [0]:
KeyJoin2 = (dfBreweryCountry.state == dfStates.code )
dfBreweryCountryState =  dfBreweryCountry.join(dfStates, KeyJoin2, 'left') \
.drop('code') \
.withColumn('state_name', coalesce('state_name', lit('Não Informado')))

dfBreweryCountryState.show()

+-----+--------------------+--------------------+-----+-------+--------------+-------------+
|   id|                name|                city|state|country|  country_name|   state_name|
+-----+--------------------+--------------------+-----+-------+--------------+-------------+
|19730|     BROUWERIJ DANNY|           ERPE-MERE|   --|     BE|       Belgium|Não Informado|
|32541|COACHELLA VALLEY ...|      THOUSAND PALMS|   CA|     US| United States|   California|
|44736|    BEEF 'O' BRADY'S|          PLANT CITY|   FL|     US| United States|      Florida|
|23372|BROADWAY WINE MER...|       OKLAHOMA CITY|   OK|     US| United States|     Oklahoma|
|35328|BRIGHTON BEER DIS...|            BRIGHTON|   GB|     GB|United Kingdom|Não Informado|
|31561|      TEDDY'S TAVERN|             SEATTLE|   WA|     US| United States|   Washington|
|35975|MODUS OPERANDI BR...|           MONA VALE|   --|     AU|     Australia|Não Informado|
| 5618|HOPS! BEER RESTAU...|       RICCIONE (RN)|   --|     IT|       

In [0]:
dfBreweryCountryState.write.mode('overwrite').csv('dbfs:/FileStore/tables/bootcampociandt/BreweriesFull')

In [0]:
keyJoin3 = (dfBreweryCountryState.id == dfBrewerryTypesAux.id)
joinFull = dfBreweryCountryState.join(dfBrewerryTypesAux, keyJoin3, 'left') \
.drop(dfBrewerryTypesAux.id)
joinFull.show()

+--------------------+--------------+-----+-------+--------------+-------------+-----+-----------+
|                name|          city|state|country|  country_name|   state_name|   id|       type|
+--------------------+--------------+-----+-------+--------------+-------------+-----+-----------+
|     BROUWERIJ DANNY|     ERPE-MERE|   --|     BE|       Belgium|Não Informado|19730|    Brewery|
|COACHELLA VALLEY ...|THOUSAND PALMS|   CA|     US| United States|   California|32541| Beer-to-go|
|COACHELLA VALLEY ...|THOUSAND PALMS|   CA|     US| United States|   California|32541|        Bar|
|COACHELLA VALLEY ...|THOUSAND PALMS|   CA|     US| United States|   California|32541|    Brewery|
|    BEEF 'O' BRADY'S|    PLANT CITY|   FL|     US| United States|      Florida|44736|     Eatery|
|    BEEF 'O' BRADY'S|    PLANT CITY|   FL|     US| United States|      Florida|44736|        Bar|
|BROADWAY WINE MER...| OKLAHOMA CITY|   OK|     US| United States|     Oklahoma|23372|      Store|
|BRIGHTON 

In [0]:
joinFull.write.mode('overwrite').csv('dbfs:/FileStore/tables/bootcampociandt/FullBreweries')

In [0]:

df = spark.table('joinFull')

display(df.select("*"))

In [0]:
dfBreweryCountryState.select("id","state").distinct().show()

In [0]:
dfBreweryCountryState.describe().show(vertical=True)

In [0]:
df_percent = dfBreweryCountryState.select("country").groupBy("country").count().withColumn('Percentual', col('count')/sum('count').over(Window.partitionBy())*100)
df_percent = df_percent.withColumn("Percentual", round(df_percent["Percentual"]))
df_percent.orderBy(df_percent.Percentual.desc()).show()

In [0]:
df_percent = dfBreweryCountryState.select("state","country").groupBy("country").count().withColumn('Percentual', col('count')/sum('count').over(Window.partitionBy())*100)
df_percent = df_percent.withColumn("Percentual", round(df_percent["Percentual"]))
df_percent.orderBy(df_percent.Percentual.desc()).show()

In [0]:
df_percent = dfBreweryCountryState.select("state").groupBy("state").count().withColumn('Percentual', col('count')/sum('count').over(Window.partitionBy())*100)
df_percent = df_percent.withColumn("Percentual", round(df_percent["Percentual"]))
df_percent.orderBy(df_percent.Percentual.desc()).show(50)

+-----+-----+----------+
|state|count|Percentual|
+-----+-----+----------+
|   --|11271|      22.0|
|   CA| 3638|       7.0|
|   GB| 2526|       5.0|
|   PA| 2454|       5.0|
|   NY| 2284|       5.0|
|   IL| 1551|       3.0|
|   OH| 1518|       3.0|
|   TX| 1342|       3.0|
|   MA| 1372|       3.0|
|   FL| 1595|       3.0|
|   NC| 1301|       3.0|
|   WI|  996|       2.0|
|   OR|  775|       2.0|
|   VA| 1036|       2.0|
|   WA| 1131|       2.0|
|   CO| 1007|       2.0|
|   MI| 1134|       2.0|
|   MN|  951|       2.0|
|   NJ|  959|       2.0|
|   MD|  670|       1.0|
|   IA|  319|       1.0|
|   CT|  692|       1.0|
|   MO|  536|       1.0|
|   ME|  380|       1.0|
|   ON|  635|       1.0|
|   TN|  422|       1.0|
|   AZ|  439|       1.0|
|   QC|  463|       1.0|
|   NH|  319|       1.0|
|   VT|  403|       1.0|
|   IN|  652|       1.0|
|   GA|  636|       1.0|
|   SC|  441|       1.0|
|   KY|  297|       1.0|
|   BC|  319|       1.0|
|   NS|   68|       0.0|
|   LA|  214|       0.0|
